In [5]:
%set_env SHELL=/bin/bash
%set_env OMP_NUM_THREADS=8
%set_env VECLIB_MAXIMUM_THREADS=1
%set_env ASE_CP2K_COMMAND=cp2k_shell.ssmp

env: SHELL=/bin/bash
env: OMP_NUM_THREADS=8
env: VECLIB_MAXIMUM_THREADS=1
env: ASE_CP2K_COMMAND=cp2k_shell.ssmp


In [10]:
# General
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# For building things
from ase import Atom, Atoms
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.build import molecule, surface, add_adsorbate, add_vacuum, sort
from ase.visualize import view
from ase.db import connect
from ase.geometry import get_layers
import nglview as nv

# Unit Conversions and Fixing Atoms
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal
from ase.constraints import FixedPlane, FixedLine, FixAtoms

# ASE Calculators
from plumed import Plumed
from ase.calculators.cp2k import CP2K
from ase.calculators.lj import LennardJones
from ase.calculators.plumed import Plumed
from ase.calculators.idealgas import IdealGas

# Geometry Optimizations and Normal Mode Analysis
from ase.optimize import LBFGS, FIRE
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

# EOS fitting for Unit Cells
from ase.eos import EquationOfState, calculate_eos

# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT

cwd = os.getcwd()

%matplotlib inline
#%reload_ext autoreload
%autosave 2


Autosaving every 2 seconds


In [11]:
# Make sure to create a softlink to MSELProjects/Python/mycalculators.py in this directory before running this command
from mycalculators import *

In [8]:
fal = read("../../Resources/xyz/furfural/FAL.xyz")
view(fal, viewer="ngl")

In [12]:
meoh = read("../../Resources/xyz/solvents/CH3OH.xyz")
view(meoh, viewer="ngl")

In [6]:
beta = read("../../Resources/cif/tmc/Beta-Mo2C_mp-1221498_conventional_standard.cif")
view(beta, viewer="ngl")

In [9]:
calc = CP2KCalculator(
    400, 
    "rVV10",
    kpoints=None,
    dipole_correction=True,
    orbital_transform=False,
    smearing=True,
    added_mos=100)

In [7]:
def SetTags(atoms, tag):
    for atom in atoms:
        atom.tag = tag

def CheckOverlap(atoms, atom_idx, rmin):
    distances = atoms.get_distances(atom_idx, range(0,natoms), mic=True)
    distances = np.delete(distances, atom_idx)
    if np.any(distances) < rmin:
        return True
    else:
        return False
    
    
def CreateProbeOnSlab(uc, probe, miller_indices, nlayers, vacuum):
    
    slab = surface(uc, miller_indices, nlayers, vacuum)
    system = slab.repeat([3,4] + [1])
    system = sort(system, system.positions[:,2])
    
    SetTags(system, 0)
    SetTags(probe, 1)
    
    box = system.get_cell()
    x = 0.5*(box[0][0] + box[1][0] + box[2][0])
    y = 0.5*(box[0][1] + box[1][1] + box[2][1])
    
    add_adsorbate(system, probe, 2.5, position=(x, y), offset=None, mol_index=0)
    
    c = FixAtoms(indices=[atom.index for atom in system if atom.position[2] < 0.5*np.sum(system.get_cell()[:,2])])
    system.set_constraint(c)
      
    return system


def CheckOverlap(atoms, atom_idx, rmin):
    distances = atoms.get_distances(atom_idx, range(0,natoms), mic=True)
    distances = np.delete(distances, atom_idx)
    if np.any(distances) < rmin:
        return True
    else:
        return False
     

        
def Translate(atoms, selected_atoms, drmax):

    dr = drmax*(np.random.rand(3)-np.array([0.5, 0.5, 0.5]))
    
    atoms_tmp = atoms[selected_atoms]
    old_position = deepcopy(atoms_tmp.positions)
    
    atoms_tmp.translate(dr)
    new_position = atoms_tmp.positions
    
    atoms[selected_atoms].positions = new_position
    atoms.wrap()
    
    return old_position
    

def MCTranslate(atoms, selected_atoms, drmax, step, updatefreq, rmin, energy = None):
    
    if energy is None:
        potential_energy_o = atoms.get_potential_energy()
        
    else:
        potential_energy_o = energy
    
    # Generate new configuration, but save old coordinates
    dr = drmax*(np.random.rand(3)-np.array([0.5, 0.5, 0.5]))
    
    atoms_tmp = atoms[selected_atoms]
    old_position = deepcopy(atoms_tmp.positions)
    
    atoms_tmp.translate(dr)
    new_position = atoms_tmp.positions
    
    atoms[selected_atoms].positions = new_position

    del atoms_tmp
    atoms.wrap()
    
    # Reject the Move if our overlap criteria is met
    if CheckOverlap(atoms,rmin):
        atoms[selected_atoms].positions = old_position
        energy = atoms.get_potential_energy()
                
    else:
        # Calculate New Potential Energy
        potential_energy_n = atoms.get_potential_energy()
        d_potential_energy = potential_energy_n - potential_energy_o
        d_potential_energy /= natoms
        
        # Accept or Reject the move according to the Metropolis Scheme
        if d_potential_energy <= 0.0:
            naccepted += 1
            energy = potential_energy_n 
        
        else:
            
            if np.random.rand() <= np.exp(-beta*d_potential_energy):
                naccepted += 1
                energy = potential_energy_n
            
            else:
                atoms[random_atom].positions = old_position
                energy = atoms.get_potential_energy()
                
    if move % maxupdate == 0:
        if naccepted/(move+1) > 0.5:
            drmax *= 1.1
        else:
            drmax *= 0.9

    

def MCRotate(atoms, selected_atoms, dthetamax, step, updatefreq, rmin, energy = None):
    
    if energy is None:
        potential_energy_o = atoms.get_potential_energy()
        
    else:
        potential_energy_o = energy
    
    # Generate new configuration, but save old coordinates
    atoms_tmp = atoms[selected_atoms]
    old_position = deepcopy(atoms_tmp.positions)
    
    rotation_vector = np.random.rand(3)
    dtheta = dthetamax*np.random.rand()  
    
    atoms_tmp.rotate(a=dtheta, v=rotation_vector, center="COP", rotate_cell=False)
    new_position = atoms_tmp.positions
    
    atoms[selected_atoms].positions = new_position
    
    del atoms_tmp
    atoms.wrap()
    
    # Reject the Move if our overlap criteria is met
    if CheckOverlap(atoms,rmin):
        atoms[random_atom].position = old_position
        energy = atoms.get_potential_energy()
                
    else:
        # Calculate New Potential Energy
        potential_energy_n = atoms.get_potential_energy()
        d_potential_energy = potential_energy_n - potential_energy_o
        d_potential_energy /= natoms
        
        # Accept or Reject the move according to the Metropolis Scheme
        if d_potential_energy <= 0.0:
            naccepted += 1
            energy = potential_energy_n 
        
        else:
            
            if np.random.rand() <= np.exp(-beta*d_potential_energy):
                naccepted += 1
                energy = potential_energy_n
            
            else:
                atoms[random_atom].position = old_position
                energy = atoms.get_potential_energy()
                
    if move % maxupdate == 0:
        if naccepted/(move+1) > 0.5:
            dthetamax *= 1.1
        else:
            dthetamax *= 0.9
            
    
    
        
def SelectFromDictionary(self, dictionary):
        chi = np.random.rand()
        lower = 0.0
        upper = 0.0
        for k, v in dictionary.items():
            upper += v
            if lower <= chi and chi <= upper:
                return k
            else:
                lower += v

                
def GetRandomSpecies(system, speciescount, solid):
    """Returns a list of indices for a random species present in the system based on the atom.tag attribute
    Assumes that solids (if present) have a tag of 0
    """
    # Pick a random tag (species), if there is a solid, pick a tag other than 0
    random_species = np.random.randint(0,speciescount)
    if solid:
        random_species += 1
        
    # Get all atoms with that tag
    return [atom.index for atom in system if atom.tag == random_species]
    
    
    
def GCMC(system, adsorbate, species_tags, solid=False, chemical_potential=-10, nsteps=10):
    '''Performs a GCMC simulation to set up an initial system with solvent molecules'''
    
    moveprobabilities={"translation":0.45, "rotation":0.45, "insertion":0.05, "deletion":0.05}
    maxdisplacements={"translation":1.0, "rotation":60.0, "insertion":None, "deletion":None}
    Naccepted={"translation":0, "rotation":0, "insertion":0, "deletion":0}
    Nattempted={"translation":0, "rotation":0, "insertion":0, "deletion":0}
    
    moleculecount = 0
    speciescount = len(species_tags)
    updatefreq = 10
    rmin = 0.75
    
    
    moveprobabilities = dict(sorted(moveprobabilities.items(), key=lambda item: item[1]))
    energy = None
    for imove in range(0, nsteps):
        # If there are molecules present, decide from the dictionary
        # Otherwise, try to insert one
        
        if not moleculecount == 0:
            
            move = SelectFromDictionary(moveprobabilities)
            
            
            if move == "translation":
                selected_atoms = GetRandomSpecies(system, speciescount, solid)
                energy = MCTranslate(atoms, selected_atoms, maxdisplacements[move], imove, updatefreq, rmin)
            
            elif move == "rotation":
                selected_atoms = GetRandomSpecies(system, speciescount, solid)
                energy = MCRotate(system)
            
            elif move == "insertion":
                
                MCInsert(system, solvent, chemical_potential)
            
            elif move == "deletion":
                MCDelete(system, solvent, chemical_potential)
            
        else:
            
            move = SelectFromDictionary(moveprobabilities)
            
            if move == "translation":
                MCTranslate(system)
            
            elif move == "rotation":
                MCRotate(system)
            
            elif move == "insertion" or "deletion":
                MCInsert(system, solvent, chemical_potential)

        print("Move, PotentialEnergy [eV], movetype, deltamax, Naccepted, AcceptanceRate")
        print(imove, energy, move, maxdisplacements[move], Naccepted, AcceptanceRate)

We have to have a way to distinguish between probe, slab, and solvent molecules within ASE. The best way to do this is with the atom.tag attribute (int). In the future, reactive moves can cause atoms to change tags and change identities. By convention, we will assume that the last key-value pair in the dictionary can fluctuate and any tag greater than the last belongs to that key type. A dictionary will be used to keep track of which atoms are what in the system. An example API could be as follows:

species_tags = {"Slab":0, "probe":1, "BA-Hydrogen":2, "Solvent":3}

1. If tag == 0, the Atom is part of the slab
2. If tag == 1, the Atom is part of the probe
3. If tag == 2, the Atom belongs to a Bronsted acid site and can react with something else. 
3. If tag >= 3, the Atom is part of a solvent molecule

In [39]:
fal = read("../../Resources/xyz/furfural/FAL.xyz")
view(fal)

<Popen: returncode: None args: ['/home/woodrowwilson/Programs/miniconda3/bin...>

In [40]:
rotation_vector = np.random.rand(3)
fal.rotate(a=60, v=rotation_vector, center="COP", rotate_cell=False)

In [41]:
view(fal)

<Popen: returncode: None args: ['/home/woodrowwilson/Programs/miniconda3/bin...>

In [17]:
   
        
def CreateProbeOnSlab(uc, probe, miller_indices, nlayers, vacuum):
    
    slab = surface(uc, miller_indices, nlayers, vacuum)
    system = slab.repeat([3,4] + [1])
    system = sort(system, system.positions[:,2])
    
    SetTags(system, 0)
    SetTags(probe, 1)
    
    box = system.get_cell()
    x = 0.5*(box[0][0] + box[1][0] + box[2][0])
    y = 0.5*(box[0][1] + box[1][1] + box[2][1])
    
    add_adsorbate(system, probe, 2.5, position=(x, y), offset=None, mol_index=0)
    
    c = FixAtoms(indices=[atom.index for atom in system if atom.position[2] < 0.5*np.sum(system.get_cell()[:,2])])
    system.set_constraint(c)
      
    return system



system = CreateProbeOnSlab(beta, fal, [1,0,1], 4, 7.5)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1


In [14]:
view(system, viewer="ngl")

In [47]:
indices = [atom.index for atom in system if atom.tag==1]
system[indices].positions

array([[ 6.67004326,  5.90822358, 17.3151135 ],
       [ 5.77823497,  6.49957763, 17.48831564],
       [ 6.88206281,  4.56017219, 17.14170231],
       [ 8.28185698,  4.397244  , 16.95971344],
       [ 8.84364817,  5.65563943, 17.03514251],
       [ 7.84468794,  6.59855222, 17.25098654],
       [ 6.12086422,  3.78691809, 17.14918627],
       [ 8.84105807,  3.48279396, 16.78853216],
       [10.2141407 ,  6.13532085, 16.95245405],
       [10.32659791,  7.2387939 , 17.03009429],
       [11.18313843,  5.38755964, 16.81087511]])

In [ ]:
system.calc = calc
system.get_potential_energy()  

In [15]:
for atom in fal:
    atom.tag = 1
    print(atom.tag)

1
1
1
1
1
1
1
1
1
1
1


In [54]:
x = np.array([2])
y = 3
Naccepted={"translation":0, "rotation":0, "insertion":0, "deletion":0}
Nattempted={"translation":0, "rotation":0, "insertion":0, "deletion":0}

def foo(x,y,Naccepted):
    x[0] +=2
    Naccepted["translation"] += 1
    return y+3

bar = foo(x,y, Naccepted)
print(bar)
print(x)
print(y)
print(Naccepted["translation"])

6
[4]
3
1
